In [ ]:
#Install roboflow if it is not there
!pip install roboflow

In [ ]:
#to download data 
from roboflow import Roboflow
rf = Roboflow(api_key="p74POuw6jj7ck2VAQpgY")
project = rf.workspace("bakchodi").project("bull-detection-project")
# version = project.version(2)bbb
dataset = version.download("coco")

In [ ]:
# to prepare labels.txt from annotation.json

import json
import os

# Define paths
dataset_path = "Bull-Detection-Project-3"  # Change this to your dataset folder
sets = ["train", "valid", "test"]

# Function to convert COCO bbox to YOLO format
def convert_bbox(size, box):
    x, y, w, h = box
    dw = 1.0 / size[0]
    dh = 1.0 / size[1]
    x_center = (x + w / 2.0) * dw
    y_center = (y + h / 2.0) * dh
    w = w * dw
    h = h * dh
    return x_center, y_center, w, h

# Process each dataset split
for dataset in sets:
    json_path = os.path.join(dataset_path, dataset, "_annotations.coco.json")
    labels_path = os.path.join(dataset_path, dataset, "labels")
    os.makedirs(labels_path, exist_ok=True)

    # Load COCO JSON
    with open(json_path, "r") as f:
        data = json.load(f)

    # Process images and annotations
    images = {img["id"]: img["file_name"] for img in data["images"]}
    for ann in data["annotations"]:
        image_id = ann["image_id"]
        category_id = ann["category_id"]
        bbox = ann["bbox"]

        if image_id in images:
            img_name = images[image_id].replace(".jpg", ".txt")
            img_path = os.path.join(labels_path, img_name)
            img_width, img_height = data["images"][image_id]["width"], data["images"][image_id]["height"]

            x_center, y_center, w, h = convert_bbox((img_width, img_height), bbox)

            # Write to YOLO format
            with open(img_path, "a") as label_file:
                label_file.write(f"{category_id} {x_center} {y_center} {w} {h}\n")


In [ ]:
# to delete cache files if it is from the datasets folders

cache_files = [
    r"C:\Users\ADMIN\Downloads\yolo-training\Bull-Detection-Project-2\train\labels.cache",
    r"C:\Users\ADMIN\Downloads\yolo-training\Bull-Detection-Project-2\test\labels.cache",
    r"C:\Users\ADMIN\Downloads\yolo-training\Bull-Detection-Project-2\valid\labels.cache"
]

for cache_path in cache_files:
    if os.path.exists(cache_path):
        os.remove(cache_path)
        print(f"🗑️ Deleted cache file: {cache_path}")

In [ ]:
import os
# do the same process to delete the unmatched images ensure to match the both images and labels count.
# repeat the below process for test and valid folders also.
train_images_path = r"C:\Users\ADMIN\Downloads\yolo-training\Bull-Detection-Project-2\train\images"
train_labels_path = r"C:\Users\ADMIN\Downloads\yolo-training\Bull-Detection-Project-2\train\labels"

train_images = set(os.listdir(train_images_path))
train_labels = set(os.listdir(train_labels_path))

# Convert labels to match image filenames
train_labels = {f.replace('.txt', '.jpg') for f in train_labels}

missing_labels = train_images - train_labels
if missing_labels:
    print("Missing label files for images:", missing_labels)
else:
    print("All train images have labels.")


In [ ]:
from ultralytics import YOLO

# Load YOLOv8 model (pre-trained weights)
model = YOLO("yolov8n.pt")

results = model.train(
    data=r"C:\Users\ADMIN\Downloads\yolo-training\Bull-Detection-Project-2\data.yaml",
    epochs=50,  # Number of epochs
    imgsz=416,  # Reduce image size for faster training
    batch=2,  # Reduce batch size to avoid memory issues
    device="cpu",  # Ensure it runs on CPU
    project="custom_yolo_saves",  # Custom folder for saving
    name="bull_detection",  # Experiment name
    save_period=10  # Save model every 10 epochs
)

In [ ]:
# preprocess and correct the format of labels.txt 

import os

# Paths to label directories
label_dirs = [
    r"C:\Users\ADMIN\Downloads\yolo-training\Bull-Detection-Project-2\train\labels",
    r"C:\Users\ADMIN\Downloads\yolo-training\Bull-Detection-Project-2\test\labels",
    r"C:\Users\ADMIN\Downloads\yolo-training\Bull-Detection-Project-2\valid\labels"
]

def clean_labels(label_dirs):
    for label_dir in label_dirs:
        for label_file in os.listdir(label_dir):
            if label_file.endswith(".txt"):
                file_path = os.path.join(label_dir, label_file)

                with open(file_path, "r") as f:
                    lines = f.readlines()

                # Remove empty files
                if not lines:
                    os.remove(file_path)
                    print(f"🗑️ Removed empty label file: {label_file}")
                    continue

                fixed_lines = []
                for line in lines:
                    parts = line.strip().split()
                    if parts and parts[0] == "1":  # Change class 1 to class 0
                        parts[0] = "0"
                    fixed_lines.append(" ".join(parts) + "\n")

                # Save the fixed label file
                with open(file_path, "w") as f:
                    f.writelines(fixed_lines)

                print(f"✅ Fixed label file: {label_file}")

# Run the function
clean_labels(label_dirs)
print("🎯 All label files are cleaned and fixed.")